In [1]:
from broprompt import Prompt
from brollm import BedrockChat, OllamaChat
model = BedrockChat()

In [2]:
tool_selector_prompt = Prompt.from_markdown("./tool_selector.md")
print(tool_selector_prompt.str)

# Tool Selection Expert

## Persona
You are a tool selection expert.

## Instructions
- Analyze the USER_INPUT carefully and select the single best matching tool from TOOLS
- The best matching tool is a tool whose name and description matches the USER_INPUT only
- Return only a tool name align with structured_output

## Caution
- Do not confuse on the tools' names and parameters if you can't find the best matching tool or not sure, return null
- Your response must be in YAML codeblock respecting the structured_output schema only

## TOOLS
{tools}

## Structured Output
**Format:** YAML codeblock

**Schema:**
```yaml
tool: tool name
```

## Examples

### Example 1
**USER_INPUT:** "add 5 and 3"

**OUTPUT:**
```yaml
tool: add
```

### Example 2
**USER_INPUT:** "clear my input"

**OUTPUT:**
```yaml
tool: clear_user_input
```

### Example 3
**USER_INPUT:** "do something and the do something is not in TOOLS"

**OUTPUT:**
```yaml
tool: null
```


In [3]:
from broprompt import list_tools, parse_codeblock_to_dict, generate_extract_parameters_prompt, validate_parameters, get_yaml_function_definition

# use <|start|><|end|> as a special token to indicate where the model should focus
def save_file(filename:str, content:str):
    """
    <|start|>This will be used only when a user ask for saving something<|end|>
    Args:
        filename (str): The name of the file to save.
        content (str): The content to write into the file.
    """
    return f"File {filename} saved with content length {len(content)}"

def add_calendar(date:str, time:str, description:str):
    """
    <|start|>This will be used only when a user ask for adding something to their calendar<|end|>
    Args:
        date (str): The date to add to the calendar in YYYY-MM-DD format.
        time (str): The time to add to the calendar in HH:MM format.
        description (str): A brief description of the event.
    """
    return f"Event '{description}' added to calendar on {date} at {time}"

tools = [save_file, add_calendar]

In [4]:
tool_selector_prompt.from_dict({"tools": list_tools(tools)})
print(tool_selector_prompt.str)

# Tool Selection Expert

## Persona
You are a tool selection expert.

## Instructions
- Analyze the USER_INPUT carefully and select the single best matching tool from TOOLS
- The best matching tool is a tool whose name and description matches the USER_INPUT only
- Return only a tool name align with structured_output

## Caution
- Do not confuse on the tools' names and parameters if you can't find the best matching tool or not sure, return null
- Your response must be in YAML codeblock respecting the structured_output schema only

## TOOLS
	- save_file: This will be used only when a user ask for saving something
	- add_calendar: This will be used only when a user ask for adding something to their calendar

## Structured Output
**Format:** YAML codeblock

**Schema:**
```yaml
tool: tool name
```

## Examples

### Example 1
**USER_INPUT:** "add 5 and 3"

**OUTPUT:**
```yaml
tool: add
```

### Example 2
**USER_INPUT:** "clear my input"

**OUTPUT:**
```yaml
tool: clear_user_input
```

### Ex

In [5]:
# message = "I'm so hapy today. Save a file"
message = "I have to visit my aunt on 2024-10-01 at 15:00. Add it to my calendar"
tool = model.run(system_prompt=tool_selector_prompt.str, messages=[model.UserMessage(message)])
tool = parse_codeblock_to_dict(tool, codeblock="yaml")
tool

{'tool': 'add_calendar'}

In [6]:
tool_call_prompt = Prompt.from_markdown("./tool_call.md")
print(tool_call_prompt.str)

# Function Argument Extractor

## Persona
You are a function argument extractor.

## Instructions
- Extract USER_INPUT required to call the function using the function's DEFINITION
- Return YAML that matches the parameters schema exactly
- Only include the keys that are required

## Structured Output
**Format:** YAML

**Schema:** Respond only with the YAML

## DEFINITION
Given the following function definition:

{definition}


In [7]:
# selected_tool = [t for t in tools if t.__name__==tool.get("tool", "nope")][0]
# tool_str = generate_extract_parameters_prompt(selected_tool)
# print(tool_str)

In [8]:
selected_tool = [t for t in tools if t.__name__==tool.get("tool", "nope")][0]
tool_str = get_yaml_function_definition(selected_tool)
print(tool_str)

Function: add_calendar(date: str, time: str, description: str)
Description: This will be used only when a user ask for adding something to their calendar

Parameters:
- date (str)
- time (str)
- description (str)

Return only the YAML format:
```yaml
date: "<value>"
time: "<value>"
description: "<value>"
```


In [9]:
tool_call_prompt.from_dict({"definition": tool_str})
print(tool_call_prompt.str)

# Function Argument Extractor

## Persona
You are a function argument extractor.

## Instructions
- Extract USER_INPUT required to call the function using the function's DEFINITION
- Return YAML that matches the parameters schema exactly
- Only include the keys that are required

## Structured Output
**Format:** YAML

**Schema:** Respond only with the YAML

## DEFINITION
Given the following function definition:

Function: add_calendar(date: str, time: str, description: str)
Description: This will be used only when a user ask for adding something to their calendar

Parameters:
- date (str)
- time (str)
- description (str)

Return only the YAML format:
```yaml
date: "<value>"
time: "<value>"
description: "<value>"
```


In [10]:
observation = model.run(tool_call_prompt.str, [model.UserMessage(message)])
print("raw", observation)
observation = parse_codeblock_to_dict(observation, codeblock="yaml")
print("parsed", observation)

raw ```yaml
date: "2024-10-01"
time: "15:00"
description: "Visit aunt"
```
parsed {'date': '2024-10-01', 'time': '15:00', 'description': 'Visit aunt'}


In [12]:
validate_parameters(observation, selected_tool)

(True, None)

In [13]:
selected_tool(**observation)

"Event 'Visit aunt' added to calendar on 2024-10-01 at 15:00"